In [ ]:
import sys
sys.path.append('../..')

from src.index_files import *
# from src.corenlp_base import Doc, Mention, Sentence
# from huggingface_hub import login
# login(os.environ['HF_TOKEN'])

In [ ]:
# dataset = QualityDataset(split='dev')
f = Factory(chunk_size=300, llm_name='meta-llama/Llama-2-7b-chat-hf')
tokenizer = AutoTokenizer.from_pretrained(f.llm_name)
# article = dataset.get_article(dataset.data[2])

### Baseline 32k QA

In [ ]:
templates = {
    "narrativeqa": "You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "qasper": "You are given a scientific article and a question. Answer the question as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nArticle: {context}\n\n Answer the question based on the above article as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "multifieldqa_en": "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "multifieldqa_zh": "阅读以下文字并用中文简短回答：\n\n{context}\n\n现在请基于上面的文章回答下面的问题，只告诉我答案，不要输出任何其他字词。\n\n问题：{input}\n回答：",
    "hotpotqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "2wikimqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "musique": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "dureader": "请基于给定的文章回答下述问题。\n\n文章：{context}\n\n请基于上述文章回答下面的问题。\n\n问题：{input}\n回答：",
    "gov_report": "You are given a report by a government agency. Write a one-page summary of the report.\n\nReport:\n{context}\n\nNow, write a one-page summary of the report.\n\nSummary:",
    "qmsum": "You are given a meeting transcript and a query containing a question or instruction. Answer the query in one or more sentences.\n\nTranscript:\n{context}\n\nNow, answer the query based on the above meeting transcript in one or more sentences.\n\nQuery: {input}\nAnswer:",
    "multi_news": "You are given several news passages. Write a one-page summary of all news. \n\nNews:\n{context}\n\nNow, write a one-page summary of all the news.\n\nSummary:",
    "vcsum": "下面有一段会议记录，请你阅读后，写一段总结，总结会议的内容。\n会议记录：\n{context}\n\n会议总结：",
    "trec": "Please determine the type of the question below. Here are some examples of questions.\n\n{context}\n{input}",
    "triviaqa": "Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{context}\n\n{input}",
    "samsum": "Summarize the dialogue into a few short sentences. The following are some examples.\n\n{context}\n\n{input}",
    "lsht": "请判断给定新闻的类别，下面是一些例子。\n\n{context}\n{input}",
    "passage_count": "There are some paragraphs below sourced from Wikipedia. Some of them may be duplicates. Please carefully read these paragraphs and determine how many unique paragraphs there are after removing duplicates. In other words, how many non-repeating paragraphs are there in total?\n\n{context}\n\nPlease enter the final count of unique paragraphs after removing duplicates. The output format should only contain the number, such as 1, 2, 3, and so on.\n\nThe final answer is: ",
    "passage_retrieval_en": "Here are 30 paragraphs from Wikipedia, along with an abstract. Please determine which paragraph the abstract is from.\n\n{context}\n\nThe following is an abstract.\n\n{input}\n\nPlease enter the number of the paragraph that the abstract is from. The answer format must be like \"Paragraph 1\", \"Paragraph 2\", etc.\n\nThe answer is: ",
    "passage_retrieval_zh": "以下是若干段落文字，以及其中一个段落的摘要。请确定给定的摘要出自哪一段。\n\n{context}\n\n下面是一个摘要\n\n{input}\n\n请输入摘要所属段落的编号。答案格式必须是\"段落1\"，\"段落2\"等格式\n\n答案是：",
    "lcc": "Please complete the code given below. \n{context}Next line of code:\n",
    "repobench-p": "Please complete the code given below. \n{context}{input}Next line of code:\n"
}

# answers = defaultdict(list)
# for task in ["narrativeqa", "qasper", "multifieldqa_en", "hotpotqa", "2wikimqa", "musique", "gov_report", "qmsum"]:
#     dataset = []
#     for sample in load_dataset('THUDM/LongBench', task, split='test'):
#         prompt = templates[task].format(context=sample['context']) if task == 'gov_report' else templates[task].format(context=sample['context'], input=sample['input'])
#         if len(tokenizer.encode(prompt)) < 32000 - 500:
#             sample['prompt'] = prompt
#             dataset.append(sample)
#     for i in tqdm(range((len(dataset) + 1) // 5)):
#         batch_samples = dataset[i*5 : (i+1)*5]
#         for sample, gen in zip(batch_samples, f.llm.generate([[HumanMessage(content=sample['prompt'])] for sample in batch_samples]).generations):
#             sample['gen'] = gen[0].text
#             sample['token_usage'] = gen[0].message.response_metadata['token_usage']
#             answers[task].append(sample)
# write_json('baseline.json', answers)

### 32k Retrieval

In [ ]:
templates = {
    "narrativeqa": "You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "qasper": "You are given a scientific article and a question. Answer the question as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nArticle: {context}\n\n Answer the question based on the above article as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "multifieldqa_en": "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "multifieldqa_zh": "阅读以下文字并用中文简短回答：\n\n{context}\n\n现在请基于上面的文章回答下面的问题，只告诉我答案，不要输出任何其他字词。\n\n问题：{input}\n回答：",
    "hotpotqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "2wikimqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "musique": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "dureader": "请基于给定的文章回答下述问题。\n\n文章：{context}\n\n请基于上述文章回答下面的问题。\n\n问题：{input}\n回答：",
    "gov_report": "You are given a report by a government agency. Write a one-page summary of the report.\n\nReport:\n{context}\n\nNow, write a one-page summary of the report.\n\nSummary:",
    "qmsum": "You are given a meeting transcript and a query containing a question or instruction. Answer the query in one or more sentences.\n\nTranscript:\n{context}\n\nNow, answer the query based on the above meeting transcript in one or more sentences.\n\nQuery: {input}\nAnswer:",
    "multi_news": "You are given several news passages. Write a one-page summary of all news. \n\nNews:\n{context}\n\nNow, write a one-page summary of all the news.\n\nSummary:",
    "vcsum": "下面有一段会议记录，请你阅读后，写一段总结，总结会议的内容。\n会议记录：\n{context}\n\n会议总结：",
    "trec": "Please determine the type of the question below. Here are some examples of questions.\n\n{context}\n{input}",
    "triviaqa": "Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{context}\n\n{input}",
    "samsum": "Summarize the dialogue into a few short sentences. The following are some examples.\n\n{context}\n\n{input}",
    "lsht": "请判断给定新闻的类别，下面是一些例子。\n\n{context}\n{input}",
    "passage_count": "There are some paragraphs below sourced from Wikipedia. Some of them may be duplicates. Please carefully read these paragraphs and determine how many unique paragraphs there are after removing duplicates. In other words, how many non-repeating paragraphs are there in total?\n\n{context}\n\nPlease enter the final count of unique paragraphs after removing duplicates. The output format should only contain the number, such as 1, 2, 3, and so on.\n\nThe final answer is: ",
    "passage_retrieval_en": "Here are 30 paragraphs from Wikipedia, along with an abstract. Please determine which paragraph the abstract is from.\n\n{context}\n\nThe following is an abstract.\n\n{input}\n\nPlease enter the number of the paragraph that the abstract is from. The answer format must be like \"Paragraph 1\", \"Paragraph 2\", etc.\n\nThe answer is: ",
    "passage_retrieval_zh": "以下是若干段落文字，以及其中一个段落的摘要。请确定给定的摘要出自哪一段。\n\n{context}\n\n下面是一个摘要\n\n{input}\n\n请输入摘要所属段落的编号。答案格式必须是\"段落1\"，\"段落2\"等格式\n\n答案是：",
    "lcc": "Please complete the code given below. \n{context}Next line of code:\n",
    "repobench-p": "Please complete the code given below. \n{context}{input}Next line of code:\n"
}

relevant_templates = {
    "narrativeqa": 'You are given a story, which can be either a novel or a movie script, a chunk of text from the story, and a question. Decide whether this chunk is relevant to the question. Do not provide any explanation.\n\nStory: {context}\n\nNow, decide whether the following chunk is relevant to the question or not. Only return "Yes" or "No" and do not provide any explanation.\n\nChunk: {chunk}\n\nQuestion: {input}\n\nAnswer:',
    "qasper": "You are given a scientific article, a chunk of text from the article and a question. Decide whether this chunk is relevant to the question. Do not provide any explanation.\n\nArticle: {context}\n\n Now, decide whether the following chunk is relevant to the question or not. Only return \"Yes\" or \"No\" and do not provide any explanation.\n\nChunk: {chunk}\n\nQuestion: {input}\n\nAnswer:",
    # "multifieldqa_en": "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "multifieldqa_zh": "阅读以下文字并用中文简短回答：\n\n{context}\n\n现在请基于上面的文章回答下面的问题，只告诉我答案，不要输出任何其他字词。\n\n问题：{input}\n回答：",
    # "hotpotqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "2wikimqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "musique": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "dureader": "请基于给定的文章回答下述问题。\n\n文章：{context}\n\n请基于上述文章回答下面的问题。\n\n问题：{input}\n回答：",
    "gov_report": "You are given a report by a government agency and a chunk of text from the report. Decide whether this chunk should be included in an extractive summary.\n\nReport:\n{context}\n\nNow, decide whether the following chunk should be included in an extractive summary. Only return \"Yes\" or \"No\" and do not provide any explanation.\n\nChunk: {chunk}\n\nAnswer:",
    "qmsum": "You are given a meeting transcript, a chunk of text from the transcript and a query containing a question or instruction. Decide whether this chunk is relevant to the query. Do not provide any explanation.\n\nTranscript:\n{context}\n\nNow, decide whether the following chunk is relevant to the query or not. Only return \"Yes\" or \"No\" and do not provide any explanation.\n\nChunk: {chunk}\n\nQuery: {input}\n\nAnswer:",
    # "multi_news": "You are given several news passages. Write a one-page summary of all news. \n\nNews:\n{context}\n\nNow, write a one-page summary of all the news.\n\nSummary:",
    # "vcsum": "下面有一段会议记录，请你阅读后，写一段总结，总结会议的内容。\n会议记录：\n{context}\n\n会议总结：",
    # "trec": "Please determine the type of the question below. Here are some examples of questions.\n\n{context}\n{input}",
    # "triviaqa": "Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{context}\n\n{input}",
    # "samsum": "Summarize the dialogue into a few short sentences. The following are some examples.\n\n{context}\n\n{input}",
    # "lsht": "请判断给定新闻的类别，下面是一些例子。\n\n{context}\n{input}",
    # "passage_count": "There are some paragraphs below sourced from Wikipedia. Some of them may be duplicates. Please carefully read these paragraphs and determine how many unique paragraphs there are after removing duplicates. In other words, how many non-repeating paragraphs are there in total?\n\n{context}\n\nPlease enter the final count of unique paragraphs after removing duplicates. The output format should only contain the number, such as 1, 2, 3, and so on.\n\nThe final answer is: ",
    # "passage_retrieval_en": "Here are 30 paragraphs from Wikipedia, along with an abstract. Please determine which paragraph the abstract is from.\n\n{context}\n\nThe following is an abstract.\n\n{input}\n\nPlease enter the number of the paragraph that the abstract is from. The answer format must be like \"Paragraph 1\", \"Paragraph 2\", etc.\n\nThe answer is: ",
    # "passage_retrieval_zh": "以下是若干段落文字，以及其中一个段落的摘要。请确定给定的摘要出自哪一段。\n\n{context}\n\n下面是一个摘要\n\n{input}\n\n请输入摘要所属段落的编号。答案格式必须是\"段落1\"，\"段落2\"等格式\n\n答案是：",
    # "lcc": "Please complete the code given below. \n{context}Next line of code:\n",
    # "repobench-p": "Please complete the code given below. \n{context}{input}Next line of code:\n"
}

answers = defaultdict(list)
for task in ["narrativeqa", "qasper", 
            #  "multifieldqa_en", "hotpotqa", "2wikimqa", "musique", 
             "gov_report", "qmsum"]:
    dataset = []
    for sample in tqdm(load_dataset('THUDM/LongBench', task, split='test')):
        prompt = templates[task].format(context=sample['context']) if task == 'gov_report' else templates[task].format(context=sample['context'], input=sample['input'])
        if len(tokenizer.encode(prompt)) < 32000 - 500:
            chunks = f.split_text(sample['context'])
            prompts = [relevant_templates[task].format(context=sample['context'], chunk=chunk) if task == 'gov_report' else relevant_templates[task].format(context=sample['context'], input=sample['input'], chunk=chunk) for chunk in chunks]
            sample['chunk_relevant'] = [(chunk, gen[0].text) for chunk, gen in zip(chunks, f.llm.generate([[HumanMessage(content=prompt)] for prompt in prompts], max_tokens=5).generations)]
            answers[task].append(sample)
            
write_json('relevant.json', answers)

### Non-context Retrieval

In [ ]:
templates = {
    "narrativeqa": "You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "qasper": "You are given a scientific article and a question. Answer the question as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nArticle: {context}\n\n Answer the question based on the above article as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "multifieldqa_en": "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "multifieldqa_zh": "阅读以下文字并用中文简短回答：\n\n{context}\n\n现在请基于上面的文章回答下面的问题，只告诉我答案，不要输出任何其他字词。\n\n问题：{input}\n回答：",
    "hotpotqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "2wikimqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "musique": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "dureader": "请基于给定的文章回答下述问题。\n\n文章：{context}\n\n请基于上述文章回答下面的问题。\n\n问题：{input}\n回答：",
    "gov_report": "You are given a report by a government agency. Write a one-page summary of the report.\n\nReport:\n{context}\n\nNow, write a one-page summary of the report.\n\nSummary:",
    "qmsum": "You are given a meeting transcript and a query containing a question or instruction. Answer the query in one or more sentences.\n\nTranscript:\n{context}\n\nNow, answer the query based on the above meeting transcript in one or more sentences.\n\nQuery: {input}\nAnswer:",
    "multi_news": "You are given several news passages. Write a one-page summary of all news. \n\nNews:\n{context}\n\nNow, write a one-page summary of all the news.\n\nSummary:",
    "vcsum": "下面有一段会议记录，请你阅读后，写一段总结，总结会议的内容。\n会议记录：\n{context}\n\n会议总结：",
    "trec": "Please determine the type of the question below. Here are some examples of questions.\n\n{context}\n{input}",
    "triviaqa": "Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{context}\n\n{input}",
    "samsum": "Summarize the dialogue into a few short sentences. The following are some examples.\n\n{context}\n\n{input}",
    "lsht": "请判断给定新闻的类别，下面是一些例子。\n\n{context}\n{input}",
    "passage_count": "There are some paragraphs below sourced from Wikipedia. Some of them may be duplicates. Please carefully read these paragraphs and determine how many unique paragraphs there are after removing duplicates. In other words, how many non-repeating paragraphs are there in total?\n\n{context}\n\nPlease enter the final count of unique paragraphs after removing duplicates. The output format should only contain the number, such as 1, 2, 3, and so on.\n\nThe final answer is: ",
    "passage_retrieval_en": "Here are 30 paragraphs from Wikipedia, along with an abstract. Please determine which paragraph the abstract is from.\n\n{context}\n\nThe following is an abstract.\n\n{input}\n\nPlease enter the number of the paragraph that the abstract is from. The answer format must be like \"Paragraph 1\", \"Paragraph 2\", etc.\n\nThe answer is: ",
    "passage_retrieval_zh": "以下是若干段落文字，以及其中一个段落的摘要。请确定给定的摘要出自哪一段。\n\n{context}\n\n下面是一个摘要\n\n{input}\n\n请输入摘要所属段落的编号。答案格式必须是\"段落1\"，\"段落2\"等格式\n\n答案是：",
    "lcc": "Please complete the code given below. \n{context}Next line of code:\n",
    "repobench-p": "Please complete the code given below. \n{context}{input}Next line of code:\n"
}

relevant_templates = {
    "narrativeqa": 'You are given a chunk of text from a story and a question. Decide whether this chunk is relevant to the question or not. Only return "Yes" or "No" and do not provide any explanation.\n\nChunk: {chunk}\n\nQuestion: {input}\n\nAnswer:',
    "qasper": "You are given a chunk of text from an article and a question. Decide whether this chunk is relevant to the question or not. Only return \"Yes\" or \"No\" and do not provide any explanation.\n\nChunk: {chunk}\n\nQuestion: {input}\n\nAnswer:",
    # "multifieldqa_en": "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "multifieldqa_zh": "阅读以下文字并用中文简短回答：\n\n{context}\n\n现在请基于上面的文章回答下面的问题，只告诉我答案，不要输出任何其他字词。\n\n问题：{input}\n回答：",
    # "hotpotqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "2wikimqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "musique": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    # "dureader": "请基于给定的文章回答下述问题。\n\n文章：{context}\n\n请基于上述文章回答下面的问题。\n\n问题：{input}\n回答：",
    # "gov_report": "You are given a report by a government agency and a chunk of text from the report. Decide whether this chunk should be included in an extractive summary.\n\nReport:\n{context}\n\nNow, decide whether the following chunk should be included in an extractive summary. Only return \"Yes\" or \"No\" and do not provide any explanation.\n\nChunk: {chunk}\n\nAnswer:",
    "qmsum": "You are given a chunk of text from a transcript and a query containing a question or instruction. Decide whether this chunk is relevant to the query or not. Only return \"Yes\" or \"No\" and do not provide any explanation.\n\nChunk: {chunk}\n\nQuery: {input}\n\nAnswer:",
    # "multi_news": "You are given several news passages. Write a one-page summary of all news. \n\nNews:\n{context}\n\nNow, write a one-page summary of all the news.\n\nSummary:",
    # "vcsum": "下面有一段会议记录，请你阅读后，写一段总结，总结会议的内容。\n会议记录：\n{context}\n\n会议总结：",
    # "trec": "Please determine the type of the question below. Here are some examples of questions.\n\n{context}\n{input}",
    # "triviaqa": "Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{context}\n\n{input}",
    # "samsum": "Summarize the dialogue into a few short sentences. The following are some examples.\n\n{context}\n\n{input}",
    # "lsht": "请判断给定新闻的类别，下面是一些例子。\n\n{context}\n{input}",
    # "passage_count": "There are some paragraphs below sourced from Wikipedia. Some of them may be duplicates. Please carefully read these paragraphs and determine how many unique paragraphs there are after removing duplicates. In other words, how many non-repeating paragraphs are there in total?\n\n{context}\n\nPlease enter the final count of unique paragraphs after removing duplicates. The output format should only contain the number, such as 1, 2, 3, and so on.\n\nThe final answer is: ",
    # "passage_retrieval_en": "Here are 30 paragraphs from Wikipedia, along with an abstract. Please determine which paragraph the abstract is from.\n\n{context}\n\nThe following is an abstract.\n\n{input}\n\nPlease enter the number of the paragraph that the abstract is from. The answer format must be like \"Paragraph 1\", \"Paragraph 2\", etc.\n\nThe answer is: ",
    # "passage_retrieval_zh": "以下是若干段落文字，以及其中一个段落的摘要。请确定给定的摘要出自哪一段。\n\n{context}\n\n下面是一个摘要\n\n{input}\n\n请输入摘要所属段落的编号。答案格式必须是\"段落1\"，\"段落2\"等格式\n\n答案是：",
    # "lcc": "Please complete the code given below. \n{context}Next line of code:\n",
    # "repobench-p": "Please complete the code given below. \n{context}{input}Next line of code:\n"
}

answers = defaultdict(list)
for task in ["narrativeqa", "qasper", 
            #  "multifieldqa_en", "hotpotqa", "2wikimqa", "musique", 
            #  "gov_report", 
             "qmsum"]:
    dataset = []
    for sample in tqdm(load_dataset('THUDM/LongBench', task, split='test')):
        prompt = templates[task].format(context=sample['context']) if task == 'gov_report' else templates[task].format(context=sample['context'], input=sample['input'])
        if len(tokenizer.encode(prompt)) < 32000 - 500:
            chunks = f.split_text(sample['context'])
            prompts = [relevant_templates[task].format(input=sample['input'], chunk=chunk) for chunk in chunks]
            sample['chunk_relevant'] = [(chunk, gen[0].text) for chunk, gen in zip(chunks, f.llm.generate([[HumanMessage(content=prompt)] for prompt in prompts], max_tokens=5).generations)]
            answers[task].append(sample)
            
write_json('relevant_non_context.json', answers)

### Answer with Retrieval

In [ ]:
templates = {
    "narrativeqa": "You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "qasper": "You are given a scientific article and a question. Answer the question as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nArticle: {context}\n\n Answer the question based on the above article as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write \"unanswerable\". If the question is a yes/no question, answer \"yes\", \"no\", or \"unanswerable\". Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:",
    "multifieldqa_en": "Read the following text and answer briefly.\n\n{context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "multifieldqa_zh": "阅读以下文字并用中文简短回答：\n\n{context}\n\n现在请基于上面的文章回答下面的问题，只告诉我答案，不要输出任何其他字词。\n\n问题：{input}\n回答：",
    "hotpotqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "2wikimqa": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "musique": "Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {input}\nAnswer:",
    "dureader": "请基于给定的文章回答下述问题。\n\n文章：{context}\n\n请基于上述文章回答下面的问题。\n\n问题：{input}\n回答：",
    "gov_report": "You are given a report by a government agency. Write a one-page summary of the report.\n\nReport:\n{context}\n\nNow, write a one-page summary of the report.\n\nSummary:",
    "qmsum": "You are given a meeting transcript and a query containing a question or instruction. Answer the query in one or more sentences.\n\nTranscript:\n{context}\n\nNow, answer the query based on the above meeting transcript in one or more sentences.\n\nQuery: {input}\nAnswer:",
    "multi_news": "You are given several news passages. Write a one-page summary of all news. \n\nNews:\n{context}\n\nNow, write a one-page summary of all the news.\n\nSummary:",
    "vcsum": "下面有一段会议记录，请你阅读后，写一段总结，总结会议的内容。\n会议记录：\n{context}\n\n会议总结：",
    "trec": "Please determine the type of the question below. Here are some examples of questions.\n\n{context}\n{input}",
    "triviaqa": "Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{context}\n\n{input}",
    "samsum": "Summarize the dialogue into a few short sentences. The following are some examples.\n\n{context}\n\n{input}",
    "lsht": "请判断给定新闻的类别，下面是一些例子。\n\n{context}\n{input}",
    "passage_count": "There are some paragraphs below sourced from Wikipedia. Some of them may be duplicates. Please carefully read these paragraphs and determine how many unique paragraphs there are after removing duplicates. In other words, how many non-repeating paragraphs are there in total?\n\n{context}\n\nPlease enter the final count of unique paragraphs after removing duplicates. The output format should only contain the number, such as 1, 2, 3, and so on.\n\nThe final answer is: ",
    "passage_retrieval_en": "Here are 30 paragraphs from Wikipedia, along with an abstract. Please determine which paragraph the abstract is from.\n\n{context}\n\nThe following is an abstract.\n\n{input}\n\nPlease enter the number of the paragraph that the abstract is from. The answer format must be like \"Paragraph 1\", \"Paragraph 2\", etc.\n\nThe answer is: ",
    "passage_retrieval_zh": "以下是若干段落文字，以及其中一个段落的摘要。请确定给定的摘要出自哪一段。\n\n{context}\n\n下面是一个摘要\n\n{input}\n\n请输入摘要所属段落的编号。答案格式必须是\"段落1\"，\"段落2\"等格式\n\n答案是：",
    "lcc": "Please complete the code given below. \n{context}Next line of code:\n",
    "repobench-p": "Please complete the code given below. \n{context}{input}Next line of code:\n"
}

answers = defaultdict(list)
relevant_non_context:dict = read_json('relevant_non_context.json')
for task, samples in relevant_non_context.items():
    dataset = []
    for sample in samples:
        context = '\n\n'.join([f'Passage {pid + 1}: {p}' for pid, (p, y_o_n) in enumerate(sample['chunk_relevant']) if 'yes' in y_o_n.lower() and 'no' not in y_o_n.lower()])
        prompt = templates[task].format(context=context) if task == 'gov_report' else templates[task].format(context=context, input=sample['input'])
        sample['prompt'] = prompt
        sample['retrieved_context'] = context
        if context:
            dataset.append(sample)
        else:
            sample['gen'] = 'unanswerable'
        answers[task].append(sample)
    for i in tqdm(range((len(dataset) + 1) // 5)):
        batch_samples = dataset[i*5 : (i+1)*5]
        for sample, gen in zip(batch_samples, f.llm.generate([[HumanMessage(content=sample['prompt'])] for sample in batch_samples]).generations):
            sample['gen'] = gen[0].text
            sample['token_usage'] = gen[0].message.response_metadata['token_usage']
write_json('relevant_non_context_answers.json', answers)


## Build Graph

In [ ]:
# narrativeqa = load_dataset('THUDM/LongBench', 'narrativeqa', split='test')
# narrativeqa = load_dataset('deepmind/narrativeqa', split='train')
qasper = load_dataset('THUDM/LongBench', 'qasper', split='test')

In [ ]:
narrativeqa[0].keys()

In [ ]:
text:str = narrativeqa[0]['document']['text']

In [ ]:
text:str = qasper[0]['context']
passages = [' '.join(p.strip().split()) for p in text.split('\n') if p.strip()]

In [ ]:
with open('article.txt', 'w') as f_out:
    f_out.write(text)

In [ ]:
with open('article.txt') as f_in:
    text = f_in.read()

In [ ]:
from wikipediaapi import Wikipedia
wiki_wiki = Wikipedia('MyProjectName (merlin@example.com)', 'en')
page_py = wiki_wiki.page('Python_(programming_language)')
text = '\n\n'.join([passage.strip() for sec_text in page_py.sections for passage in sec_text.full_text().splitlines()])

In [ ]:
passages = [' '.join(p.strip().split()) for p in text.split('\n\n') if p.strip()]

In [ ]:
passages[16]

In [ ]:
f.nlp.pipeline

In [ ]:
# passages = passages[17:]
chunks, start_p = list[str](), 0
for end_p in range(len(passages)):
    passage_text = '\n\n'.join(passages[start_p : end_p+1])
    # passage_len = len(tokenizer.encode(passage_text, add_special_tokens=False))
    passage_len = len(list(f.nlp(passage_text, disable=['lemmatizer', 'ner']).sents))
    if passage_len < 3 and end_p+1 < len(passages):
        continue
    start_p = end_p+1
    chunks.append(passage_text)

In [ ]:
topic_switch_prompt = '''
Identify the first sentence in the passages below that marks a turning point in the narrative focus. This sentence should act as a boundary, where the content before it maintains a consistent focus on one topic, and the content after it shifts to a different focus. In your explanation, use one or two sentences to briefly describe the narrative focus both before and immediately after this turning point.


Passages:

{passages}


Now, generate your response, organizing it under the following headers: "Turning Point" for the first turning point sentence, "Topic before" and "Topic after" for the explanation of topic before and immediately after turning point sentence.
'''

In [ ]:
topic_switch_prompt = '''
Identify the first sentence in the passages below that marks a turning point in the narrative focus. This sentence should act as a boundary, where the content before it maintains a consistent focus on one topic, and the content after it shifts to a different focus. In your response, first, briefly describe the narrative focus at the beginning, second, identify the turning point sentence and finally, briefly describe the narrative focus immediately after the turning point.


Passages:

{passages}


Now, generate your response, organizing it under the following headers: "Beginning Topic" for the topic at the beginning, "Turning Point" for the first turning point sentence, and "Topic after" for the explanation of topic immediately after the turning point sentence.
'''

In [ ]:
query_topic_prompt = '''
What is the topic of the following passage?

{passage}

Now, generate your response with one statement starting with "Current Topic: ".
'''

In [ ]:
query_topic_consist_prompt = '''
Using the provided background context, current topic, recent passages under the current topic, and the next passage in the narrative, evaluate how the next passage influences the current topic. Leverage the background context to deepen your understanding of the topics and passage content.

Background Context:
{context}

Current Topic:
{topic}

Recent Passages:
{last_passage}

You will now receive the next passage. Assess its impact on the current topic in relation to the recent passages and choose the condition that best describes the effect:

If the next passage stays within the current topic and adds further information, respond with "Same Topic".
If the next passage introduces new information that shifts away from the current topic, respond with a concise statement of the new topic starting with "New Topic: ".
If the next passage explores a specific aspect in greater detail to enhance understanding of the current topic, respond with a concise statement of the subtopic starting with "Subtopic: ".

Next Passage:
{passage}

Briefly explain your thought first and finally give me your response as described above.
'''

In [ ]:
query_topic_consist_prompt = '''
Using the provided background context, current topic, recent passages under the current topic, and the next passage in the narrative, evaluate how the next passage influences the current topic. Leverage the background context to deepen your understanding of the topics and passage content.

Background Context:
{context}

Current Topic:
{topic}

Recent Passages:
{last_passage}

You will now receive the next passage. Assess its impact on the current topic in relation to the recent passages using the following guidelines:

1.	Compare Keywords and Main Ideas:
    If keywords and main ideas are similar, the passages likely share the same topic or a subtopic.
    If they differ significantly, the next passage may introduce a new topic.
2.	Check Transitional Words and Topic Sentences:
    Look for transitions like “however” or “meanwhile,” which might signal a shift.
    The first sentence of the next passage can indicate if it's continuing the previous topic or introducing something new.
3.	Examine Narrative Flow and Context:
    If the characters or setting change, it might indicate a new topic.
    Consistent elements usually suggest the same topic or a subtopic.
4.	Identify the Author's Purpose:
    A shift in the author's focus or purpose often signals a new topic.
    A deeper exploration of the same idea usually indicates a subtopic.
5.	Look at Structural Cues:
    A new paragraph with a different focus may introduce a new topic.
    Continuation within the same context suggests the same topic or a subtopic.

Next Passage:
{passage}

If the next passage stays with the same topic, respond with "Same Topic".
If the next passage introduces a new topic, respond with a concise statement of the new topic starting with "New Topic: ".
If the next passage explores a subtopic of the current topic, respond with a concise statement of the subtopic starting with "Subtopic: ".
Briefly explain your thought first and finally give me your response as described above.
'''

In [ ]:
# passages = [f.nlp(p, disable=['ner']) for p in passages[17:]]

In [ ]:
class Topic:
    def __init__(self, tid:int, parent_pid:int, topic:str, passage:str):
        self.tid = tid
        self.parent_pid = parent_pid
        self.topic = topic
        self.passages = list[str]()
        self.structure = list[str|Topic]()
        self.passages.append(passage)
        self.structure.append(passage)
        self.parent: Topic = None
    
    def add_subtopic(self, subtopic:str, passage:str):
        subtopic_cnt = 0
        for p in self.structure:
            if isinstance(p, Topic):
                subtopic_cnt += 1
                
        subtopic_obj = Topic(tid=subtopic_cnt, parent_pid=len(self.passages)-1, topic=subtopic, passage=passage)
        subtopic_obj.parent = self
        self.structure.append(subtopic_obj)
        return subtopic_obj
    
    def append_passage(self, passage:str):
        self.passages.append(passage)
        self.structure.append(passage)
        
    def get_context(self, n:int=3, context_width:int=1):
        current_topic = self
        context = []
        layer_cnt = 0
        while current_topic.parent is not None and layer_cnt < n:
            # context.insert(0, f'{current_topic.parent.get_formated_topic()}\n\n{current_topic.parent.passages[current_topic.parent_pid]}')
            # context = current_topic.parent.passages[max(0, current_topic.parent_pid + 1 - context_width) : current_topic.parent_pid + 1] + context
            context.insert(0, current_topic.parent.passages[current_topic.parent_pid])
            current_topic = current_topic.parent
            layer_cnt += 1
        return '\n\n'.join(context)
    
    def get_formated_topic(self):
        accumulated_tid_str = '.'.join([str(tid + 1) for tid in self.get_accumulated_tid()])
        return f'Topic {accumulated_tid_str}. {self.topic}'
    
    def get_accumulated_tid(self):
        tids = list[int]()
        current_topic = self
        while current_topic is not None:
            tids.insert(0, current_topic.tid)
            current_topic = current_topic.parent
        return tids

In [ ]:
def query_topic(passage:str):
    prompt = query_topic_prompt.format(passage=passage)
    response = f.llm.generate([[HumanMessage(prompt)]]).generations[0][0].text
    
    start_topic = response.split('Current Topic: ')[-1].strip()
    current_topic_obj = Topic(tid=0, parent_pid=-1, topic=start_topic, passage=passage)
    return current_topic_obj, [(prompt, response)]
        
def query_topic_consist(current_topic_obj:Topic, passage:str):
    prompt_responses = list[tuple[str, str]]()
    
    prompt = query_topic_consist_prompt.format(context=current_topic_obj.get_context(), topic=current_topic_obj.get_formated_topic(), last_passage=current_topic_obj.passages[-1], passage=passage)
    response = f.llm.generate([[HumanMessage(prompt)]]).generations[0][0].text
    prompt_responses.append((prompt, response))
    
    if 'Same Topic' in response:
        current_topic_obj.append_passage(passage)
    elif 'New Topic:' in response:
        if current_topic_obj.parent_pid < 0:
            new_topic = sent_tokenize([line for line in response.split('New Topic:')[-1].strip().splitlines() if line][0])[0]
            current_topic_obj = Topic(tid=current_topic_obj.tid + 1, parent_pid=-1, topic=new_topic, passage=passage)
        else:
            current_topic_obj, temp_prompt_responses = query_topic_consist(current_topic_obj.parent, passage)
            prompt_responses.extend(temp_prompt_responses)
    elif 'Subtopic:' in response:
        subtopic = sent_tokenize([line for line in response.split('Subtopic:')[-1].strip().splitlines() if line][0])[0]
        current_topic_obj = current_topic_obj.add_subtopic(subtopic, passage)
    else:
        print(prompt)
        print('----------------------------------------------')
        print(response)
        raise NotImplementedError
    return current_topic_obj, prompt_responses

In [ ]:
topics, prompt_responses = list[Topic](), []
for cid, chunk in enumerate(tqdm(chunks)):
    if not topics:
        current_topic_obj, temp_prompt_responses = query_topic(chunk)
        topics.append(current_topic_obj)
    else:
        current_topic_obj, temp_prompt_responses = query_topic_consist(current_topic_obj, chunk)
        if current_topic_obj.parent_pid < 0 and current_topic_obj.tid != topics[-1].tid:
            topics.append(current_topic_obj)
    prompt_responses.extend(temp_prompt_responses)
    last_passage = passage_text
    # if cid >= 30:
    #     break

In [ ]:
import pickle
with open('article.pickle', 'wb') as f_out:
    pickle.dump(topics, f_out)

In [ ]:
test_id = 3
print(prompt_responses[test_id][0])
print('----------------------------------------------')
print(prompt_responses[test_id][1])

In [ ]:
def get_menu(structure:List[str|Topic], expand_subtopics:List[str]=[]):
    menu, topic_mask = list[str](), list[bool]()
    for p in structure:
        if isinstance(p, str):
            menu.append(p)
            topic_mask.append(False)
        else:
            formated_topic = p.get_formated_topic()
            menu.append(formated_topic)
            topic_mask.append(True)
            if any([formated_topic.startswith(expand_subtopic) for expand_subtopic in expand_subtopics]):
                temp_menu, temp_topic_mask = get_menu(p.structure, expand_subtopics)
                menu.extend(temp_menu)
                topic_mask.extend(temp_topic_mask)
    return menu, topic_mask

def plot_menu(menu:List[str], topic_mask:List[bool]):
    for p, is_topic in zip(menu, topic_mask):
        if is_topic:
            print('------------------------------------------')
        print(p, '\n')

In [ ]:
with open('python.pickle', 'rb') as f_in:
    topics:List[Topic] = pickle.load(f_in)

In [ ]:
plot_menu(*get_menu(topics, [
    # 'Topic 2. ',
    # 'Topic 1. ',
    # 'Topic 3. ',
    # 'Topic 4. ',
    # 'Topic 5. ',
    # 'Topic 8. '
]))

In [ ]:
print('\n'.join([f'Section {tid+1}. {t.topic}' for tid, t in enumerate(topics)]))

In [ ]:
topics[2].topic

In [ ]:
topics[2].structure

In [ ]:
start_p, start_sent = 0, 0
for end_p in range(len(passages)):
    temp_passages = [p.text for p in passages[start_p : end_p+1]]
    if start_sent > 0:
        temp_passages = [''.join([sent.text_with_ws for sent in passages[start_p].sents])] + temp_passages[1:]
    passage_text = '\n\n'.join(temp_passages)
    passage_len = len(tokenizer.encode(passage_text, add_special_tokens=False))
    if passage_len < 2000 and end_p+1 < len(passages):
        continue
    response = f.llm.generate([[HumanMessage(topic_switch_prompt.format(passages=temp_passages))]])
    break

In [ ]:
print(response.generations[0][0].text)

In [ ]:
print(response.generations[0][0].text)

In [ ]:
print(passage_text)

In [ ]:
print('\n\n'.join(passages[0:5]))

In [ ]:
examine_passages = passages[:3]

In [ ]:
background_event_cls_prompt = '''
Given a piece of text, decide whether it is background knowledge or dynamic knowledge.

Background knowledge describing facts that are true in most contexts.'''

## Test

In [ ]:
from metrics import (
    qa_f1_score,
    rouge_zh_score,
    qa_f1_zh_score,
    rouge_score,
    classification_score,
    retrieval_score,
    retrieval_zh_score,
    count_score,
    code_sim_score,
)

dataset2metric = {
    "narrativeqa": qa_f1_score,
    "qasper": qa_f1_score,
    "multifieldqa_en": qa_f1_score,
    "multifieldqa_zh": qa_f1_zh_score,
    "hotpotqa": qa_f1_score,
    "2wikimqa": qa_f1_score,
    "musique": qa_f1_score,
    "dureader": rouge_zh_score,
    "gov_report": rouge_score,
    "qmsum": rouge_score,
    "multi_news": rouge_score,
    "vcsum": rouge_zh_score,
    "trec": classification_score,
    "triviaqa": qa_f1_score,
    "samsum": rouge_score,
    "lsht": classification_score,
    "passage_retrieval_en": retrieval_score,
    "passage_count": count_score,
    "passage_retrieval_zh": retrieval_zh_score,
    "lcc": code_sim_score,
    "repobench-p": code_sim_score,
}

In [ ]:
baseline_samples = read_json('baseline.json')
baseline_samples.keys()

In [ ]:
len(baseline_samples['qmsum'])

In [ ]:
baseline_samples = read_json('baseline.json')
for task, samples in baseline_samples.items():
    for sample in samples:
        if 'Unanswerable' not in sample['answers']:
            sample['score'] = max(dataset2metric[task](sample['gen'], ground_truth) for ground_truth in sample['answers'])
        else:
            sample['score'] = -1
    print(task, np.mean([sample['score'] for sample in samples if sample['score'] != -1]))

In [ ]:
[sample['answers'] for sample in baseline_samples['qasper']]

In [ ]:
relevant_non_context_samples = read_json('relevant_non_context_answers.json')
for task, samples in relevant_non_context_samples.items():
    for sample in samples:
        if 'Unanswerable' not in sample['answers']:
            sample['score'] = max(dataset2metric[task](sample['gen'], ground_truth) for ground_truth in sample['answers'])
        else:
            sample['score'] = -1
    print(task, np.mean([sample['score'] for sample in samples if sample['score'] != -1]))

In [ ]:
relevant_samples = read_json('relevant_answers.json')
for task, samples in relevant_samples.items():
    for sample in samples:
        if 'Unanswerable' not in sample['answers']:
            sample['score'] = max(dataset2metric[task](sample['gen'], ground_truth) for ground_truth in sample['answers'])
        else:
            sample['score'] = -1
    print(task, np.mean([sample['score'] for sample in samples if sample['score'] != -1]))

In [ ]:
compare_list = defaultdict(list)
for task in ['narrativeqa', 'qasper', 'gov_report', 'qmsum']:
    for sample_b in baseline_samples[task]:
        for sample_r in relevant_samples[task]:
            if sample_b['context'] == sample_r['context'] and sample_b['input'] == sample_r['input']:
                compare_list[task].append({'base': sample_b, 'ret': sample_r})
                break

In [ ]:
task = 'narrativeqa'
bad_compare_list = [p for p in compare_list[task] if p['base']['score'] > p['ret']['score']]
print('bad', len(bad_compare_list))
good_compare_list = [p for p in compare_list[task] if p['base']['score'] < p['ret']['score']]
print('good', len(good_compare_list))

In [ ]:
narrativeqa_check = [5,6,7,8,10,11,17,19,21]

In [ ]:
compare_pair = bad_compare_list[6]

In [ ]:
compare_pair['base']

In [ ]:
compare_pair['ret']

In [ ]:
print(compare_pair['ret']['context'])

In [ ]:
compare_pair['ret'].keys()

In [ ]:
print(compare_pair['ret']['retrieved_context'])

In [ ]:
for chunk, r in compare_pair['ret']['chunk_relevant']:
    print(chunk)
    print('---------------------------')
    print(r)
    print('---------------------------')
    print()